In [4]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT = 'http://spamassassin.apache.org/old/publiccorpus/'
HAM_URL = DOWNLOAD_ROOT + '20030228_easy_ham.tar.bz2'
SPAM_URL = DOWNLOAD_ROOT + '20030228_spam.tar.bz2'
SPAM_PATH = os.path.join('datasets', 'spam')

def fetch_spam_data(ham_url=HAM_URL, spam_url=SPAM_URL, spam_path=SPAM_PATH):
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
    for filename, url in (('ham.tar.bz2', ham_url), ('spam.tar.bz2', spam_url)):
        tgz_path = os.path.join(spam_path, filename)
        if not os.path.isfile(tgz_path):
            urllib.request.urlretrieve(url, tgz_path)
        data_tgz = tarfile.open(tgz_path)
        data_tgz.extractall(path=spam_path)
        data_tgz.close()

fetch_spam_data()

In [68]:
HAM_DIR = os.path.join(SPAM_PATH, 'easy_ham')
SPAM_DIR = os.path.join(SPAM_PATH, 'spam')

ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

print(len(ham_filenames), len(spam_filenames))

2500 500


In [19]:
import email
import email.policy

def load_email(is_spam, filename, spam_path=SPAM_PATH):
    dir = 'spam' if is_spam else 'easy_ham'
    with open(os.path.join(spam_path, dir, filename), 'rb') as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

In [20]:
print(han_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [43]:
print(spam_emails[1].get_content().strip())

1) Fight The Risk of Cancer!
http://www.adclick.ws/p.cfm?o=315&s=pk007

2) Slim Down - Guaranteed to lose 10-12 lbs in 30 days
http://www.adclick.ws/p.cfm?o=249&s=pk007

3) Get the Child Support You Deserve - Free Legal Advice
http://www.adclick.ws/p.cfm?o=245&s=pk002

4) Join the Web's Fastest Growing Singles Community
http://www.adclick.ws/p.cfm?o=259&s=pk007

5) Start Your Private Photo Album Online!
http://www.adclick.ws/p.cfm?o=283&s=pk007

Have a Wonderful Day,
Offer Manager
PrizeMama













If you wish to leave this list please use the link below.
http://www.qves.com/trim/?ilug@linux.ie%7C17%7C114258


-- 
Irish Linux Users' Group: ilug@linux.ie
http://www.linux.ie/mailman/listinfo/ilug for (un)subscription information.
List maintainer: listmaster@linux.ie


In [25]:
# 이메일에 이미지나 첨부파일을 가진 multipart 찾기
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return 'multipart ({})'.format(', '.join([get_email_structure(sub_email) for sub_email in payload]))
        
    else:
        return email.get_content_type()

In [137]:
# a = Counter()
# l = ['a', 'a', 'b', 'c', 'c', 'c']
# for ll in l:
#     a[ll] += 1
# print(a.most_common())

[('c', 3), ('a', 2), ('b', 1)]


In [128]:
from collections import Counter

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [129]:
structures_counter(ham_emails).most_common() # 햄은 plain text가 많고

[('text/plain', 2408),
 ('multipart (text/plain, application/pgp-signature)', 66),
 ('multipart (text/plain, text/html)', 8),
 ('multipart (text/plain, text/plain)', 4),
 ('multipart (text/plain)', 3),
 ('multipart (text/plain, application/octet-stream)', 2),
 ('multipart (text/plain, text/enriched)', 1),
 ('multipart (text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart (multipart (text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart (text/plain, video/mng)', 1),
 ('multipart (text/plain, multipart (text/plain))', 1),
 ('multipart (text/plain, application/x-pkcs7-signature)', 1),
 ('multipart (text/plain, multipart (text/plain, text/plain), text/rfc822-headers)',
  1),
 ('multipart (text/plain, multipart (text/plain, text/plain), multipart (multipart (text/plain, application/x-pkcs7-signature)))',
  1),
 ('multipart (text/plain, application/x-java-applet)', 1)]

In [30]:
structures_counter(spam_emails).most_common() # 스팸은 html일 경우가 많다

[('text/plain', 218),
 ('text/html', 183),
 ('multipart (text/plain, text/html)', 45),
 ('multipart (text/html)', 20),
 ('multipart (text/plain)', 19),
 ('multipart (multipart (text/html))', 5),
 ('multipart (text/plain, image/jpeg)', 3),
 ('multipart (text/html, application/octet-stream)', 2),
 ('multipart (text/plain, application/octet-stream)', 1),
 ('multipart (text/html, text/plain)', 1),
 ('multipart (multipart (text/html), application/octet-stream, image/jpeg)',
  1),
 ('multipart (multipart (text/plain, text/html), image/gif)', 1),
 ('multipart/alternative', 1)]

In [40]:
for header, value in spam_emails[0].items():
    print(header, ':', value)

Return-Path : <12a1mailbot1@web.de>
Delivered-To : zzzz@localhost.spamassassin.taint.org
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received : from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received : from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From : 12a1mailbot1@web.de
Received : from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To : dcek1a1@netsgo.com
Subject : Life Insurance - Why Pay More?
Date : Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version : 1.0
Message-ID : <0103c1042001882DD_IT7@dd_it7>
Content-Type : text/html; charset="iso-8859-1"
Content-Transfer-Encoding : qu

In [41]:
spam_emails[0]['Subject']

'Life Insurance - Why Pay More?'

In [46]:
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(ham_emails + spam_emails)
y = np.array(len(han_emails)*[0] + len(spam_emails)*[1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<ipython-input-46-064ecb43f04b>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array(ham_emails + spam_emails)


In [53]:
import re
from html import unescape

def html_to_plain_text(html):
    txt = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    txt = re.sub('<a\s.*?>', ' HYPERLINK ', txt, flags=re.M | re.S | re.I)
    txt = re.sub('<.*?>', '', txt, flags=re.M | re.S)
    txt = re.sub(r'(\s*\n)+', '\n', txt, flags=re.M | re.S)
    
    return unescape(txt)

In [63]:
html_spam_emails = [email for email in X_train[y_train==1] if get_email_structure(email) == 'text/html']

sample_html_spam = html_spam_emails[7]
print(sample_html_spam.get_content().strip()[:1000], '...')
# html 스팸 원본

<HTML><HEAD><TITLE></TITLE><META http-equiv="Content-Type" content="text/html; charset=windows-1252"><STYLE>A:link {TEX-DECORATION: none}A:active {TEXT-DECORATION: none}A:visited {TEXT-DECORATION: none}A:hover {COLOR: #0033ff; TEXT-DECORATION: underline}</STYLE><META content="MSHTML 6.00.2713.1100" name="GENERATOR"></HEAD>
<BODY text="#000000" vLink="#0033ff" link="#0033ff" bgColor="#CCCC99"><TABLE borderColor="#660000" cellSpacing="0" cellPadding="0" border="0" width="100%"><TR><TD bgColor="#CCCC99" valign="top" colspan="2" height="27">
<font size="6" face="Arial, Helvetica, sans-serif" color="#660000">
<b>OTC</b></font></TD></TR><TR><TD height="2" bgcolor="#6a694f">
<font size="5" face="Times New Roman, Times, serif" color="#FFFFFF">
<b>&nbsp;Newsletter</b></font></TD><TD height="2" bgcolor="#6a694f"><div align="right"><font color="#FFFFFF">
<b>Discover Tomorrow's Winners&nbsp;</b></font></div></TD></TR><TR><TD height="25" colspan="2" bgcolor="#CCCC99"><table width="100%" border="0" 

In [64]:
print(html_to_plain_text(sample_html_spam.get_content())[:1000], '...')
# html 제거한 스팸


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Watch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future.
Put CBYI on your watch list, acquire a position TODAY.
REASONS TO INVEST IN CBYI
A profitable company and is on track to beat ALL earnings estimates!
One of the FASTEST growing distributors in environmental & safety equipment instruments.
Excellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research.
RAPIDLY GROWING INDUSTRY
Industry revenues exceed $900 million, estimates indicate that there could be as much as $25 billi

In [65]:
# 포멧에 상관없이 이메일을 입력으로 받아서 일반 텍스트를 출력하는 함수
def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ('type/plain', 'text/html'):
            continue
        try:
            content = part.get_content()
        except:
            content = str(part.get_payload())
        
        if ctype == 'text/plain':
            return content
        else:
            html = content
    
    if html:
        return html_to_plain_text(html)